# Convert all sports dfs into one

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime, time
today = pd.Timestamp.today().date()

import pytz

## Read

In [2]:
# Define a custom date parser for the corresponding formats
fb_date_parser = lambda x: pd.to_datetime(x, format="%d/%m/%y").date()
nba_nhl_date_parser = lambda x: pd.to_datetime(x, format="%a, %b %d, %Y").date()


# Function for home/away cols for nba/nhl
def assign_teams(df, home_away_col, user_team_col, opponent_col):
    df['Home Team'] = np.where(df[home_away_col] == 'vs.', df[user_team_col], df[opponent_col])
    df['Away Team'] = np.where(df[home_away_col] == '@', df[user_team_col], df[opponent_col])
    return df


In [8]:
# Read in files with their dates
## football/soccer
df_fb = pd.read_csv(
    r"C:\Users\Owner\Documents\Data Projects\GitHub\Apps\project_w\2024_10_28__all_in_one\data\df_fb_master_2024_10_28.csv",
    parse_dates=['Date'], 
    date_parser=fb_date_parser
)
### Create/convert cols as necessary
df_fb['Date'] = pd.to_datetime(df_fb['Date']).dt.date
df_fb = df_fb[df_fb['Date'] >= today]
df_fb['Time'] = df_fb['Time'].astype(str).str.replace(r'\s+', '', regex=True)
df_fb['Time'] = df_fb['Time'].replace('-', pd.NA)
df_fb['Time'] = pd.to_datetime(df_fb['Time'], format='%H:%M').dt.time


## nba
df_nba = pd.read_csv(
    r"C:\Users\Owner\Documents\Data Projects\GitHub\Apps\project_w\2024_10_28__all_in_one\data\df_nba_master_2024_10_29.csv",
    parse_dates=['Date'], 
    date_parser=nba_nhl_date_parser
)
### Create/convert cols as necessary
df_nba['Date'] = pd.to_datetime(df_nba['Date']).dt.date
df_nba = assign_teams(df_nba, 'Home/Away', 'user_team', 'Opponent')
df_nba['Start (ET)'] = pd.to_datetime(df_nba['Start (ET)'], infer_datetime_format=True).dt.time
df_nba = df_nba.rename(columns={'Start (ET)': 'Time'})
df_nba['Time'] = pd.to_datetime(df_nba['Time'].astype(str)) - pd.to_timedelta(2, unit='hours')
df_nba['Time'] = df_nba['Time'].dt.time


## nhl
df_nhl = pd.read_csv(
    r"C:\Users\Owner\Documents\Data Projects\GitHub\Apps\project_w\2024_10_28__all_in_one\data\df_nhl_master_2024_10_30.csv",
    parse_dates=['Date'], 
    date_parser=nba_nhl_date_parser
)
### Create/convert cols as necessary
df_nhl['Date'] = pd.to_datetime(df_nhl['Date']).dt.date
df_nhl = assign_teams(df_nhl, 'Home/Away', 'user_team', 'Opponent')
df_nhl['Time'] = pd.to_datetime(df_nhl['Time'], infer_datetime_format=True).dt.time
df_nhl['Time'] = pd.to_datetime(df_nhl['Time'].astype(str)) - pd.to_timedelta(2, unit='hours')
df_nhl['Time'] = df_nhl['Time'].dt.time

## Organize

In [4]:
df_fb.head()

,Team,Date,League,Home Team,Time,Away Team
26,Arsenal,2024-11-02,PRL,Newcastle United,06:30:00,Arsenal
27,Arsenal,2024-11-06,UCL,Inter Milan,13:00:00,Arsenal
28,Arsenal,2024-11-10,PRL,Chelsea,09:30:00,Arsenal
29,Arsenal,2024-11-23,PRL,Arsenal,08:00:00,Nottingham Forest
30,Arsenal,2024-11-26,UCL,Sporting Lisbon,13:00:00,Arsenal


In [5]:
df_fb[df_fb['Date'] >= today].head()

,Team,Date,League,Home Team,Time,Away Team
26,Arsenal,2024-11-02,PRL,Newcastle United,06:30:00,Arsenal
27,Arsenal,2024-11-06,UCL,Inter Milan,13:00:00,Arsenal
28,Arsenal,2024-11-10,PRL,Chelsea,09:30:00,Arsenal
29,Arsenal,2024-11-23,PRL,Arsenal,08:00:00,Nottingham Forest
30,Arsenal,2024-11-26,UCL,Sporting Lisbon,13:00:00,Arsenal


In [ ]:
# Desired cols: user_team, date, league/game type, home_team, away_team, time
# nba_cols = ['user_team','Date','game_type','Home Team','Away Team','Time']
df_nba[df_nba['Date'] >= today].head()

,G,Date,Time,Home/Away,Opponent,Result,OT,Tm,Opp,W,...,Streak,Attend.,LOG,Notes,computer_date,user_team,GameTimeStatus,game_type,Home Team,Away Team
5,6,2024-11-01,17:30:00,vs.,Sacramento Kings,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2024-11-01,Atlanta Hawks,Upcoming,regular season,Atlanta Hawks,Sacramento Kings
6,7,2024-11-03,17:00:00,@,New Orleans Pelicans,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2024-11-03,Atlanta Hawks,Upcoming,regular season,New Orleans Pelicans,Atlanta Hawks
7,8,2024-11-04,17:45:00,vs.,Boston Celtics,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2024-11-04,Atlanta Hawks,Upcoming,regular season,Atlanta Hawks,Boston Celtics
8,9,2024-11-06,17:30:00,vs.,New York Knicks,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2024-11-06,Atlanta Hawks,Upcoming,regular season,Atlanta Hawks,New York Knicks
9,10,2024-11-08,17:00:00,@,Detroit Pistons,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2024-11-08,Atlanta Hawks,Upcoming,regular season,Detroit Pistons,Atlanta Hawks


In [ ]:
# nhl_cols = ['user_team','Date','game_type','Home Team','Away Team','Time']
df_nhl[df_nhl['Date'] >= today].head()

,GP,Date,Time,Home/Away,Opponent,GF,GA,Result,OT,W,...,Streak,Att.,LOG,Notes,computer_date,user_team,GameTimeStatus,game_type,Home Team,Away Team
9,10,2024-10-31,17:00:00,@,Pittsburgh Penguins,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2024-10-31,Anaheim Ducks,Upcoming,regular season,Pittsburgh Penguins,Anaheim Ducks
10,11,2024-11-03,18:00:00,vs.,Chicago Blackhawks,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2024-11-03,Anaheim Ducks,Upcoming,regular season,Anaheim Ducks,Chicago Blackhawks
11,12,2024-11-05,20:00:00,vs.,Vancouver Canucks,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2024-11-05,Anaheim Ducks,Upcoming,regular season,Anaheim Ducks,Vancouver Canucks
12,13,2024-11-08,20:00:00,vs.,Minnesota Wild,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2024-11-08,Anaheim Ducks,Upcoming,regular season,Anaheim Ducks,Minnesota Wild
13,14,2024-11-10,18:00:00,vs.,Columbus Blue Jackets,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2024-11-10,Anaheim Ducks,Upcoming,regular season,Anaheim Ducks,Columbus Blue Jackets


## Combine fb, nba, nhl

In [20]:
# First, rename and reorder columns in df_fb
df_fb_fin = df_fb.rename(columns={'Team': 'user_team', 'League': 'game_type'})
df_fb_fin = df_fb_fin[['user_team', 'Date', 'game_type', 'Home Team', 'Away Team', 'Time']].copy()
df_fb_fin['Sport'] = 'Soccer'

# Select and reorder columns in df_nba and df_nhl
df_nba_fin = df_nba[['user_team', 'Date', 'game_type', 'Home Team', 'Away Team', 'Time']].copy()
df_nba_fin['Sport'] = 'NBA'

df_nhl_fin = df_nhl[['user_team', 'Date', 'game_type', 'Home Team', 'Away Team', 'Time']].copy()
df_nhl_fin['Sport'] = 'NHL'

In [25]:
# Stack all three dataframes and Sort the combined dataframe by Date and then by Time, both in ascending order
df_fin_all = pd.concat([df_fb_fin, df_nba_fin, df_nhl_fin], ignore_index=True).sort_values(by=['Date', 'Time', 'Home Team'], ascending=[True, True, True]).reset_index(drop=True)

In [26]:
df_fin_future = df_fin_all[df_fin_all['Date'] >= today]
df_fin_future

,user_team,Date,game_type,Home Team,Away Team,Time,Sport
452,Fiorentina,2024-10-31,SEA,Genoa,Fiorentina,11:30:00,Soccer
453,Getafe,2024-10-31,CDR,Manises,Getafe,12:00:00,Soccer
454,Espanyol,2024-10-31,CDR,San Tirso,Espanyol,12:00:00,Soccer
455,Atletico Madrid,2024-10-31,CDR,Vic,Atletico Madrid,12:00:00,Soccer
456,Lazio,2024-10-31,SEA,Como,Lazio,13:45:00,Soccer
...,...,...,...,...,...,...,...
7400,Toulouse,2025-05-17,LI1,Saint-Etienne,Toulouse,NaT,Soccer
7401,Strasbourg,2025-05-17,LI1,Strasbourg,Le Havre,NaT,Soccer
7402,FC Cologne,2025-05-18,2.B,FC Cologne,Kaiserslautern,07:30:00,Soccer
7403,Hertha Berlin,2025-05-18,2.B,Hertha Berlin,Hannover 96,07:30:00,Soccer


In [27]:
df_fin_future[df_fin_future['Sport'].isin(['NBA','NHL'])]

,user_team,Date,game_type,Home Team,Away Team,Time,Sport
461,Boston Bruins,2024-10-31,regular season,Carolina Hurricanes,Boston Bruins,17:00:00,NHL
462,Carolina Hurricanes,2024-10-31,regular season,Carolina Hurricanes,Boston Bruins,17:00:00,NHL
463,Philadelphia Flyers,2024-10-31,regular season,Philadelphia Flyers,St. Louis Blues,17:00:00,NHL
464,St. Louis Blues,2024-10-31,regular season,Philadelphia Flyers,St. Louis Blues,17:00:00,NHL
465,Anaheim Ducks,2024-10-31,regular season,Pittsburgh Penguins,Anaheim Ducks,17:00:00,NHL
...,...,...,...,...,...,...,...
7216,Ottawa Senators,2025-04-17,regular season,Ottawa Senators,Carolina Hurricanes,17:00:00,NHL
7217,Pittsburgh Penguins,2025-04-17,regular season,Pittsburgh Penguins,Washington Capitals,17:00:00,NHL
7218,Washington Capitals,2025-04-17,regular season,Pittsburgh Penguins,Washington Capitals,17:00:00,NHL
7219,Detroit Red Wings,2025-04-17,regular season,Toronto Maple Leafs,Detroit Red Wings,17:00:00,NHL


In [28]:
df_fin_future[df_fin_future['Sport'] == 'NBA']

,user_team,Date,game_type,Home Team,Away Team,Time,Sport
471,Memphis Grizzlies,2024-10-31,regular season,Memphis Grizzlies,Milwaukee Bucks,18:00:00,NBA
472,Milwaukee Bucks,2024-10-31,regular season,Memphis Grizzlies,Milwaukee Bucks,18:00:00,NBA
475,Dallas Mavericks,2024-10-31,regular season,Dallas Mavericks,Houston Rockets,18:30:00,NBA
476,Houston Rockets,2024-10-31,regular season,Dallas Mavericks,Houston Rockets,18:30:00,NBA
477,San Antonio Spurs,2024-10-31,regular season,Utah Jazz,San Antonio Spurs,19:00:00,NBA
...,...,...,...,...,...,...,...
7148,Portland Trail Blazers,2025-04-13,regular season,Portland Trail Blazers,Los Angeles Lakers,13:30:00,NBA
7149,Sacramento Kings,2025-04-13,regular season,Sacramento Kings,Phoenix Suns,13:30:00,NBA
7150,Phoenix Suns,2025-04-13,regular season,Sacramento Kings,Phoenix Suns,13:30:00,NBA
7151,San Antonio Spurs,2025-04-13,regular season,San Antonio Spurs,Toronto Raptors,13:30:00,NBA
